<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/Cadec_twadr_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
base_dir = 'medical-concept-normalization'

In [ ]:
os.mkdir(base_dir)

In [ ]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [ ]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [ ]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [ ]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [ ]:
path = Path('medical-concept-normalization/data_collection')

In [ ]:
# training data set

train_csv = path/'TwADR-L.fold-7.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [ ]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-7.validation.csv",header=None)
valid.head()

,0,1
0,13,insomniac
1,24,racing heart
2,60,cigarette withdrawal
3,91,jitters
4,93,shooting


In [ ]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-7.test.csv",header=None)
test.head()

,0,1
0,13,keeping me awake
1,94,anxiety
2,119,incr anxiety
3,119,anxiety
4,121,itching


In [ ]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-cadec.csv')

In [ ]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [ ]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [ ]:
data_clas.add_test(test,label=0)


In [ ]:
data_lm.save()
data_clas.save()

In [ ]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'the',
 'and',
 'to',
 'my',
 'pain',
 'of',
 'in',
 'a',
 'it',
 'me',
 'mood',
 'increased',
 'for',
 'have',
 'was',
 '-',
 ')',
 '(',
 'drug',
 'lipitor',
 'this',
 'disorder',
 'on',
 'abnormal',
 'withdrawal',
 'not',
 'symptoms',
 'anxiety',
 'had',
 'that',
 'is',
 'muscle',
 'with',
 'weight',
 'like',
 'but',
 'decreased',
 "n't",
 'feeling',
 'taking',
 'disorders',
 'behavior',
 'sleeplessness',
 'after',
 'up',
 'syndrome',
 'sleep',
 'got',
 'insomnia',
 'so',
 'blood',
 'diseases',
 'all',
 'feel',
 'side',
 'am',
 'from',
 'finding',
 'disturbance',
 'somnolence',
 'back',
 'depression',
 'cholesterol',
 'energy',
 '!',
 'effects',
 'day',
 'loss',
 'mental',
 'be',
 'been',
 'body',
 '/',
 'skin',
 'heart',
 'gain',
 'as',
 'acute',
 'nausea',
 'appetite',
 'take',
 'no',
 'about',
 'at',
 'dizziness',
 'off',
 'now',
 'mg',
 'has',
 'fatigue',
 'out',
 'do',
 '

In [ ]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.301860,3.447126,0.429548,09:25


In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.270081,3.013472,0.486635,09:26


In [ ]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.056666,2.976197,0.491071,09:20


In [ ]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.017321,2.946982,0.494531,09:18


In [ ]:
learn.save_encoder('TwADR-cadec_fold7_first')

In [ ]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-cadec_fold7_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.177192,6.548279,0.056338,00:23


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.245907,6.570937,0.105634,00:27


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.721471,5.037560,0.140845,01:05


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.444561,4.537719,0.225352,01:10
1,5.209799,4.408756,0.239437,01:12


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.929905,4.235127,0.253521,01:11
1,4.754183,4.128231,0.288732,01:06


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.561887,3.925485,0.281690,01:09
1,4.400131,3.965159,0.309859,01:10


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.191002,3.796646,0.295775,01:05
1,4.035128,3.789541,0.309859,01:06


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.866744,3.660089,0.309859,01:12
1,3.719861,3.595023,0.309859,01:08


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.541752,3.485255,0.352113,01:12
1,3.458559,3.447229,0.387324,01:12
2,3.285770,3.326516,0.380282,01:04
3,3.117748,3.339302,0.394366,01:05


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.962584,3.278273,0.429577,01:05
1,2.874791,3.305027,0.415493,01:11
2,2.699726,3.243513,0.415493,01:08
3,2.543836,3.226346,0.422535,01:07


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.371392,3.186740,0.436620,01:12
1,2.393822,3.256419,0.408451,01:09
2,2.222875,3.188813,0.443662,01:08
3,2.060678,3.180953,0.429577,01:11


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.951220,3.176714,0.436620,01:07
1,1.916722,3.202010,0.436620,01:12
2,1.767669,3.137132,0.443662,01:07
3,1.620170,3.168604,0.450704,01:08


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.526865,3.184197,0.450704,01:05
1,1.521385,3.283867,0.464789,01:10
2,1.397128,3.243200,0.471831,01:10
3,1.277366,3.256428,0.471831,01:03


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.180284,3.286139,0.478873,01:03
1,1.209558,3.274119,0.457746,01:06
2,1.113189,3.322426,0.464789,01:10
3,1.018587,3.326386,0.457746,01:08


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.934403,3.366995,0.457746,01:07
1,0.964353,3.466676,0.457746,01:08
2,0.915766,3.382648,0.436620,01:04
3,0.857747,3.413395,0.450704,01:13


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.766535,3.351264,0.443662,01:04
1,0.746372,3.370763,0.450704,01:07


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.710865,3.568898,0.464789,01:09
1,0.680068,3.651678,0.457746,01:10


In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.651936,3.581722,0.443662,01:06


In [ ]:
# save the best model

learn.save_encoder('TwADR-cadec_fold7')

In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.629988,3.629579,0.450704,01:06


In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.637981,3.634472,0.457746,01:07


In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.595152,3.591135,0.443662,01:06


In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.615485,3.590800,0.443662,01:13


In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.557999,3.731659,0.457746,01:05


# Part three: Predict on the test dataset

In [ ]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[1787, 978, 978, 978, 195, 249, 154, 2048, 189, 189, 2149, 649, 231, 231, 1936, 2147, 2147, 2147, 233, 233, 249, 857, 300, 1054, 393, 1136, 418, 426, 518, 1214, 1787, 1711, 1099, 749, 749, 749, 749, 791, 1545, 1545, 818, 824, 2168, 1884, 852, 852, 1279, 1062, 974, 1356, 1062, 441, 978, 978, 1518, 233, 426, 1718, 1669, 1099, 1136, 1175, 1265, 1545, 1062, 61, 195, 1308, 1321, 1327, 1099, 1507, 1873, 974, 1432, 1806, 1701, 1526, 1545, 1573, 1163, 1597, 1597, 1597, 1597, 1597, 1597, 1597, 1597, 1597, 1610, 1610, 108, 947, 1073, 1787, 1531, 1767, 1718, 1718, 1753, 947, 1718, 1718, 1718, 561, 1597, 1787, 825, 1073, 1787, 1787, 1974, 1099, 1787, 1529, 1373, 1915, 1936, 1936, 1936, 1925, 749, 1283, 1942, 1942, 2113, 16, 1265, 2048, 1813, 61, 418, 1769, 1718, 2119, 1718, 2147, 2147, 2191, 1597, 1099]
[13, 94, 119, 119, 121, 143, 154, 189, 189, 189, 224, 230, 231, 231, 233, 233, 233, 233, 233, 233, 249, 265, 300, 359, 393, 396, 418, 426, 518, 596, 693, 712, 744, 749, 749, 749, 749, 791, 

In [ ]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

59
0.4154929577464789
